In [21]:
import re
from typing import List

import pandas as pd
import torch
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
all_data_text = pd.read_csv('data/data.csv')

In [6]:
all_data_text.head()

,genre,title,author,link,text
0,миниатюры,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов,https://www.proza.ru/2025/07/19/1660,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,миниатюры,Нас не догонят,Нас не догонят,https://www.proza.ru/2025/07/19/1653,Правая нога словно приросла к педали газа. Стр...
2,миниатюры,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану,https://www.proza.ru/2025/07/19/1644,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,миниатюры,Новая кврейская поговорка,Новая кврейская поговорка,https://www.proza.ru/2025/07/19/1641,"Всё, что дороже, чем бесплатно - дорого!"
4,миниатюры,Космическая Свита Примадонны,Космическая Свита Примадонны,https://www.proza.ru/2025/07/19/1630,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


In [7]:
all_data_text = all_data_text.drop(columns=["genre", "author", "link"])

In [8]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


In [13]:
all_data_text = all_data_text.dropna().reset_index(drop=True)

In [15]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [22]:
def tokenize(text: str) -> List[str]:
    tokens = word_tokenize(text)
    return tokens

In [17]:
all_data_text["text"] = all_data_text["text"].apply(clean_text)

In [18]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,скромный подиум кристаллов иные камни без прим...
1,Нас не догонят,правая нога словно приросла к педали газа стре...
2,Снежный шар. 200. Не по плану,снежный шар 200 не по плану солнце давно укати...
3,Новая кврейская поговорка,всё что дороже чем бесплатно дорого
4,Космическая Свита Примадонны,космическая свита примадонны вокруг земли лета...


In [23]:
all_data_text["text"] = all_data_text["text"].apply(tokenize)

In [24]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, бе..."
1,Нас не догонят,"[правая, нога, словно, приросла, к, педали, га..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, не, по, плану, солнце, дав..."
3,Новая кврейская поговорка,"[всё, что, дороже, чем, бесплатно, дорого]"
4,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."


In [27]:
all_data_text = all_data_text[all_data_text["text"].apply(len) >= 30]

In [28]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,"[скромный, подиум, кристаллов, иные, камни, бе..."
1,Нас не догонят,"[правая, нога, словно, приросла, к, педали, га..."
2,Снежный шар. 200. Не по плану,"[снежный, шар, 200, не, по, плану, солнце, дав..."
4,Космическая Свита Примадонны,"[космическая, свита, примадонны, вокруг, земли..."
5,Боровая кантата,"[пасхальное, о, это, божественный, лес, где, с..."


In [29]:
len(all_data_text)

14566

In [30]:
all_data_text = all_data_text.reset_index(drop=True)

In [31]:
all_data_text.to_csv("data/data_tokenize.csv", index=False)